In [1]:
# References
# https://towardsdatascience.com/complete-image-augmentation-in-opencv-31a6b02694f5
# https://theailearner.com/2019/05/07/add-different-noise-to-an-image/
# https://www.kaggle.com/parulpandey/overview-of-popular-image-augmentation-packages

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
import random

In [61]:
data_dir = "data_mip3/train/no_connection"
out_dir = "data_mip3_aug/train/no_connection"

In [4]:
def rotation(img, angle):
    #angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))
    return img

In [5]:
def add_light(image, gamma):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    image=cv2.LUT(image, table)
    return image

In [6]:
def gaussian_blur(image, rad, blur):
    image = cv2.GaussianBlur(image,(rad,rad),blur)
    return image

In [7]:
def gaussian_noise(img, amount):
    # Generate Gaussian noise
    gauss = np.random.normal(0,amount,img.size)
    gauss = gauss.reshape(img.shape[0],img.shape[1]).astype('uint8')
    # Add the Gaussian noise to the image
    img_gauss = cv2.add(img,gauss)
    return img_gauss

In [8]:
def specle_noise(img, std):
    gauss = np.random.normal(0,std,img.size)
    gauss = gauss.reshape(img.shape[0],img.shape[1]).astype('uint8')
    noise = img + img * gauss
    return noise

In [12]:
def contrast(img, contrast):
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        img = cv2.addWeighted(img, alpha_c, img, 0, gamma_c)
    return(img)

In [62]:
def augment(img):
    # Perform augmentation of image
    
    # Perform uniform transformations (ut) of the image.
    # Uniform transformations are the transformations where all pixels are treated uniformly.
    img_ut = []
    
    # Original
    img_ut.append(img)
    
    # Flip
    img_ut.append(cv2.flip(img, 0))# Vertical
    img_ut.append(cv2.flip(img, 1))# Horizontal
    
    # Rotate 90 degrees
    #img_ut.append(cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE))
    #img_ut.append(cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))

    # Rotate 5, 10, 15, 20, 25, 30  degrees
    #for angle in [10, 20, 30]:
    #    img_ut.append(rotation(img, angle))

    # Add light (change brightness)
    img_ut.append(add_light(img, 2))
    img_ut.append(add_light(img, 3))
    img_ut.append(add_light(img, 0.5))
    img_ut.append(add_light(img, 0.3))

    # Change contrast
    img_ut.append(contrast(img, 50))
    img_ut.append(contrast(img, 70))

    # Perform non-uniform transformations (nut) on all the uniform tranform images
    img_nut = []
    for image in img_ut:
        # Guassian blur
        img_nut.append(gaussian_blur(image, 3, 0))
        #img_nut.append(gaussian_blur(image, 5, 0)) # This is too much bluring

        # Noise
        img_nut.append(gaussian_noise(image, 0.3))
        #img_nut.append(gaussian_noise(image, 0.4)) # This may be too much noise
        img_nut.append(specle_noise(image, 0.3))
        #img_nut.append(specle_noise(image, 0.4)) # This may be too much noise

    #print(len(img_ut), len(img_nut))
    return(img_ut+img_nut)


In [63]:
# Run augmentation on all files in the data_dir
cnt = 0
for filename in os.listdir(data_dir):
    #print(filename)
    img = cv2.imread(data_dir+"/"+filename, 0)
    #print(img.shape)
    images = augment(img)
    #print("Total number of augmented images =", len(images))
    fname = filename.replace(".png","")
    i = 0
    for image in images:
        cv2.imwrite(out_dir + "/" + fname + "_" + str(i) + ".png", image)
        i += 1
    cnt += 1
    #break
print("Total number of images processed:", cnt)

Total number of images processed: 17010


In [225]:
# Check augmented images
#plt.figure(figsize=(50, 50))
#for i in range(0, len(img_augmented)):
#    ax = plt.subplot(11, 10, i+1)
#    #print(images[i].shape)
#    plt.imshow(img_augmented[i], cmap=plt.get_cmap("gray"))
#    plt.axis("off")
